# HOG Gradient Filter 

In [1]:
import cv2
import numpy as np

def compute_hog_descriptor(img, cell_size=8, block_size=2, bins=9):
    """
    Compute the HOG descriptor for a given grayscale image.
    
    Args:
        img: 2D np.array (grayscale image)
        cell_size: number of pixels per cell (int)
        block_size: number of cells per block (int)
        bins: number of angle bins (e.g., 9 for 0~180°)
        
    Returns:
        1D np.array: normalized HOG descriptor
    """
    # 1. Gradient computation (Sobel)
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=1)

    magnitude = np.sqrt(gx**2 + gy**2)
    angle = np.arctan2(gy, gx) * (180 / np.pi) % 180  # 0~180도 범위

    h, w = img.shape
    cells_x = w // cell_size
    cells_y = h // cell_size

    # 2. Create a histogram for each cell
    hist_tensor = np.zeros((cells_y, cells_x, bins), dtype=np.float32)
    bin_width = 180 // bins

    for i in range(cells_y):
        for j in range(cells_x):
            mag_cell = magnitude[i*cell_size:(i+1)*cell_size,
                                 j*cell_size:(j+1)*cell_size]
            ang_cell = angle[i*cell_size:(i+1)*cell_size,
                             j*cell_size:(j+1)*cell_size]
            
            hist = np.zeros(bins, dtype=np.float32)
            for y in range(cell_size):
                for x in range(cell_size):
                    bin_idx = int(ang_cell[y, x] // bin_width) % bins
                    hist[bin_idx] += mag_cell[y, x]
            hist_tensor[i, j] = hist

    # 3. Block-level normalization (L2-Hys)
    blocks_y = cells_y - block_size + 1
    blocks_x = cells_x - block_size + 1
    hog_vector = []

    for y in range(blocks_y):
        for x in range(blocks_x):
            block = hist_tensor[y:y+block_size, x:x+block_size, :].ravel()
            eps = 1e-6
            norm = np.sqrt(np.sum(block**2) + eps**2)
            normalized = block / norm
            # L2-Hys: clipping + normalization
            normalized = np.clip(normalized, 0, 0.2)
            normalized /= np.sqrt(np.sum(normalized**2) + eps**2)
            hog_vector.extend(normalized)

    return np.array(hog_vector)


In [2]:
import matplotlib.pyplot as plt

# read image (grayscale)
img = cv2.imread('../Source files/cat-825302_1920.jpg', cv2.IMREAD_GRAYSCALE)

# (Option) sqaure crop
h, w = img.shape
side = min(h, w)
img_cropped = img[0:side, 0:side]

# HOG computation
hog_vec = compute_hog_descriptor(img_cropped)

print(f'HOG descriptor shape: {hog_vec.shape}')


HOG descriptor shape: (646416,)
